In [39]:
import numpy as np
import trimesh
import tqdm 

In [40]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    # -------- TODO -----------
    # S=1/2*|A x B|
    vector1 = vertices[faces[:, 1],:] - vertices[faces[:, 0],:]
    vector2 = vertices[faces[:, 2],:] - vertices[faces[:, 0],:]
    area=0.5*np.linalg.norm(np.cross(vector1, vector2), axis=1)
    prob = area/np.sum(area)
    sampled_face = np.random.choice(len(faces), sample_num, p=prob)
    #三角形sample，sample a,b,分a+b>1和a+b<1两种情况，用a,b表示三角形内的点
    a=np.random.rand(sample_num)
    b=np.random.rand(sample_num)
    vec0=vertices[faces[sampled_face, 0],:]
    special_id=np.where(a+b>1)
    a[special_id]=1-a[special_id]
    b[special_id]=1-b[special_id]
    a=a.reshape(-1,1)
    b=b.reshape(-1,1)
    uniform_pc = vec0 + a*vector1[ sampled_face,:] + b*vector2[sampled_face,:]
    return area, prob, uniform_pc
        

In [41]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    # 先随机取一个点，随后每次取离已有点最远的点
    # -------- TODO -----------
    results = []
    results.append(pc[np.random.randint(len(pc))])
    dist = np.linalg.norm(pc - results[0], axis=1)
    for i in range(1, sample_num):
        farthest_id = np.argmax(dist)
        results.append(pc[farthest_id])
        dist = np.minimum(dist, np.linalg.norm(pc - pc[farthest_id], axis=1))
    results=np.array(results)
    return results


In [42]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}


np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [43]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)
# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

In [44]:
# task 3: metrics
from earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------

CD_mean = 0
CD_var = 0
EMD_mean = 0
EMD_var = 0


try_num=5
CD=np.zeros(try_num)
EMD=np.zeros(try_num)
CD1=np.zeros(try_num)
EMD1=np.zeros(try_num)
for i in range (try_num):
    _,_,uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)
    
    _,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
    farthest_pc = farthest_point_sampling(tmp_pc, final_sample_num)
    dis=np.linalg.norm(np.reshape(uniform_pc, (uniform_pc.shape[0], 1, 3)) - farthest_pc, axis=2)
    CD[i] = (np.average(np.amin(dis, axis=1)) + np.average(np.amin(dis, axis=0))) * 0.5 
    uniform_pc=list(uniform_pc)
    for j in range(len(uniform_pc)):
        uniform_pc[j]=tuple(uniform_pc[j])
    farthest_pc=list(farthest_pc)
    for j in range(len(farthest_pc)):
        farthest_pc[j]=tuple(farthest_pc[j])
    
    EMD[i] = earthmover_distance(list(uniform_pc), list(farthest_pc))

CD_mean = np.mean(CD)
CD_var = np.var(CD)
EMD_mean = np.mean(EMD)
EMD_var = np.var(EMD)


# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

move 0.001953125 dirt from (52.96274267737481, 27.90707930591452, 35.048014550853864) to (52.16901736315478, 30.220433574694525, 33.12199934485261) for a cost of 0.006080195267786785
move 0.001953125 dirt from (39.91594495221886, 33.91978583677575, 26.711722554897495) to (38.6248683946073, 34.2419484310799, 26.648656779421938) for a cost of 0.0026018709250382106
move 0.001953125 dirt from (29.5634895161758, 30.333321950953657, 21.980554718385754) to (28.9366512009219, 30.280423064887508, 22.005651546067185) for a cost of 0.0012296227504287453
move 0.001953125 dirt from (14.457090386417438, 37.73361148878776, 41.093957009331525) to (13.82334962564174, 38.501362188827656, 40.96122926768127) for a cost of 0.0019615882482500757
move 0.001953125 dirt from (32.76849266465852, 15.044220757956866, 37.03842396499235) to (29.203683406404068, 14.572104378319567, 34.26125574349278) for a cost of 0.008874029886478155
move 0.001953125 dirt from (24.52559049965189, 17.07901759329915, 32.1432355335420

In [45]:
CD_mean = np.mean(CD)
CD_var = np.var(CD)
EMD_mean = np.mean(EMD)
EMD_var = np.var(EMD)
print(CD_mean, CD_var, EMD_mean, EMD_var)

1.3573316114210698 9.210506300602593e-05 2.566395591983818 0.07490250277914175
